In [37]:
#%pip install scikit-learn

  Using cached scikit_learn-1.7.1-cp311-cp311-macosx_12_0_arm64.whl.metadata (11 kB)
  Using cached scipy-1.16.1-cp311-cp311-macosx_14_0_arm64.whl.metadata (61 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
Using cached scikit_learn-1.7.1-cp311-cp311-macosx_12_0_arm64.whl (8.7 MB)
Using cached joblib-1.5.2-py3-none-any.whl (308 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 209.2 kB/s  0:02:15m0:00:0100:04
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [scikit-learn] [scikit-learn]
Note: you may need to restart the kernel to use updated packages.


In [56]:
#!pip install osmnx

In [2]:
# Complete OSMnx-based VRP solution
import osmnx as ox
import folium
import numpy as np
import networkx as nx
from math import radians, sin, cos, sqrt, atan2
#

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [58]:
#%pip uninstall ortools -y
#%pip install ortools folium

In [3]:
from ortools.constraint_solver import pywrapcp, routing_enums_pb2

### Locations and Vehicles

In [4]:
# Our delivery locations in Lahore
locations = [
    ("Warehouse", 31.5204, 74.3587),  # central Lahore (depot)
    ("Shop A",    31.5340, 74.3572),
    ("Shop B",    31.5157, 74.3673),
    ("Shop C",    31.5280, 74.3810),
    ("Shop D",    31.5060, 74.3430),
    ("Shop E",    31.5400, 74.3440),
    ("Shop F",    31.5220, 74.3320),
    ("Shop G",    31.5100, 74.3790),
    ("Shop H",    31.5380, 74.3720),
]

# Demands (warehouse = 0, shops need fertilizer bags)
demands = [0, 40, 20, 30, 10, 12, 15, 18, 20]


print("📍 Setup complete:")
print(f"  Locations: {len(locations)}")
print(f"  Total demand: {sum(demands)} bags")


📍 Setup complete:
  Locations: 9
  Total demand: 165 bags


Real data - Time to validate

In [5]:

all_locations = [
    ("Warehouse", 31.522334, 72.576500),
    ('Shop 1', 31.570639, 72.889028),
    ('Shop 2', 31.591778, 72.836139),
    ('Shop 3', 31.495, 72.85875),
    ('Shop 4', 31.653, 72.865278),
    ('Shop 5', 31.677139, 72.909444),
    ('Shop 6', 31.480528, 72.652944),
    ('Shop 7', 31.431111, 72.745694),
    ('Shop 8', 31.644861, 72.818),
    ('Shop 9', 31.643861, 72.636389),
    ('Shop 10', 31.613639, 72.455833),
    ('Shop 11', 31.613806, 72.750056),
    ('Shop 12', 31.431444, 72.496167),
    ('Shop 13', 31.392056, 72.77875),
    ('Shop 14', 31.454667, 72.798083),
    ('Shop 15', 31.636528, 72.503222),
    ('Shop 16', 31.597333, 72.611944),
    ('Shop 17', 31.536472, 72.535139),
    ('Shop 18', 31.482861, 72.705389),
    ('Shop 19', 31.57825, 72.667722),
    ('Shop 20', 31.659056, 72.539472),
]

all_demands = [0, 40, 20, 30, 10, 12, 15, 18, 20, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Filter out locations with zero demand (except warehouse at index 0)
locations = []
demands = []

for i, (location, demand) in enumerate(zip(all_locations, all_demands)):
    if i == 0 or demand > 0:  # Keep warehouse (index 0) and locations with demand > 0
        locations.append(location)
        demands.append(demand)

print("📍 Setup complete:")
print(f"  Total locations: {len(all_locations)}")
print(f"  Active locations (with demand): {len(locations)}")
print(f"  Filtered out: {len(all_locations) - len(locations)} locations")
print(f"  Total demand: {sum(demands)} bags")

print("\n📦 Active locations:")
for i, (name, _, _) in enumerate(locations):
    print(f"  {name}: {demands[i]} bags")

📍 Setup complete:
  Total locations: 21
  Active locations (with demand): 9
  Filtered out: 12 locations
  Total demand: 165 bags

📦 Active locations:
  Warehouse: 0 bags
  Shop 1: 40 bags
  Shop 2: 20 bags
  Shop 3: 30 bags
  Shop 4: 10 bags
  Shop 5: 12 bags
  Shop 6: 15 bags
  Shop 7: 18 bags
  Shop 8: 20 bags


In [6]:
len(demands)

9

In [7]:
# Enhanced vehicle fleet with rental and owned vehicles
vehicles = [
    {
        "count": 1,
        "capacity": 200,
        "type": "owned",
        "cost_per_km": 100.0,  # Simple cost per km for owned vehicles
    },
        {
        "count": 1,
        "capacity": 100,
        "type": "owned",
        "cost_per_km": 40.0,# Cost per km after limit exceeded
    },
    {
        "count": 2,
        "capacity": 80,
        "type": "rental",
        "rental_cost": 5000.0,     # Fixed daily rental cost
        "km_limit": 50,            # Free km included in rental
        "excess_cost_per_km": 80.0 # Cost per km after limit exceeded
    },
    {
        "count": 1,
        "capacity": 50,
        "type": "rental",
        "rental_cost": 3000.0,     # Fixed daily rental cost
        "km_limit": 40,            # Free km included in rental
        "excess_cost_per_km": 60.0 # Cost per km after limit exceeded
    }
]

print("\n🚛 Enhanced vehicle fleet:")
for i, v in enumerate(vehicles):
    if v["type"] == "owned":
        print(f"  {v['count']}x {v['capacity']}-bag OWNED truck(s) @ ₨{v['cost_per_km']}/km")
    else:
        print(f"  {v['count']}x {v['capacity']}-bag RENTAL truck(s):")
        print(f"    • Daily rental: ₨{v['rental_cost']}")
        print(f"    • Free km limit: {v['km_limit']} km")
        print(f"    • Excess cost: ₨{v['excess_cost_per_km']}/km")


🚛 Enhanced vehicle fleet:
  1x 200-bag OWNED truck(s) @ ₨100.0/km
  1x 100-bag OWNED truck(s) @ ₨40.0/km
  2x 80-bag RENTAL truck(s):
    • Daily rental: ₨5000.0
    • Free km limit: 50 km
    • Excess cost: ₨80.0/km
  1x 50-bag RENTAL truck(s):
    • Daily rental: ₨3000.0
    • Free km limit: 40 km
    • Excess cost: ₨60.0/km


In [8]:
# Download road network using the method that worked
def download_road_network_safe(locations):
    """Download road network using smaller area method"""
    try:
        # Use center point method (which worked for you)
        center_lat = sum(loc[1] for loc in locations) / len(locations)
        center_lon = sum(loc[2] for loc in locations) / len(locations)

        print(f"📍 Downloading network around center: ({center_lat:.4f}, {center_lon:.4f})")

        # Download with 5km radius (adjust if needed)
        G = ox.graph_from_point((center_lat, center_lon), dist=5000, network_type='drive')

        print(f"✅ SUCCESS! Network downloaded:")
        print(f"   Nodes: {len(G.nodes)}")
        print(f"   Edges: {len(G.edges)}")

        return G, True

    except Exception as e:
        print(f"❌ Network download failed: {e}")
        return None, False

# Download the network
G, network_success = download_road_network_safe(locations)

📍 Downloading network around center: (31.5629, 72.7946)
✅ SUCCESS! Network downloaded:
   Nodes: 52
   Edges: 134


In [9]:
def haversine_km(lat1, lon1, lat2, lon2):
    """Fallback distance calculation"""
    R = 6371.0
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    return R * c

def calculate_distance_matrix(locations, graph=None):
    """Calculate distance matrix using OSMnx or fallback to haversine"""
    n = len(locations)
    dist_matrix = np.zeros((n, n))

    if graph is not None:
        print("🛣️ Calculating real road distances...")

        # Find nearest nodes for each location
        nearest_nodes = []
        for i, (name, lat, lon) in enumerate(locations):
            try:
                node = ox.nearest_nodes(graph, lon, lat)
                nearest_nodes.append(node)
                print(f"   {name}: found node {node}")
            except Exception as e:
                print(f"   ❌ {name}: error - {e}")
                nearest_nodes.append(None)

        # Calculate distances
        for i in range(n):
            for j in range(n):
                if i == j:
                    dist_matrix[i][j] = 0
                elif nearest_nodes[i] and nearest_nodes[j]:
                    try:
                        # Real road distance
                        path = nx.shortest_path(graph, nearest_nodes[i], nearest_nodes[j], weight='length')
                        path_length = sum(ox.utils_graph.get_route_edge_attributes(graph, path, 'length'))
                        dist_matrix[i][j] = path_length / 1000  # Convert to km
                    except:
                        # Fallback to haversine
                        dist_matrix[i][j] = haversine_km(locations[i][1], locations[i][2],
                                                       locations[j][1], locations[j][2]) * 1.25
                else:
                    # Fallback to haversine
                    dist_matrix[i][j] = haversine_km(locations[i][1], locations[i][2],
                                                   locations[j][1], locations[j][2]) * 1.25

        print("✅ Real road distances calculated!")

    else:
        print("🔄 Using haversine distances...")
        for i in range(n):
            for j in range(n):
                if i == j:
                    dist_matrix[i][j] = 0
                else:
                    dist_matrix[i][j] = haversine_km(locations[i][1], locations[i][2],
                                                   locations[j][1], locations[j][2]) * 1.25

    return dist_matrix

# Calculate distances
distance_matrix = calculate_distance_matrix(locations, G if network_success else None)

🛣️ Calculating real road distances...
   Warehouse: found node 2904594039
   Shop 1: found node 7507164078
   Shop 2: found node 6647010475
   Shop 3: found node 6655363527
   Shop 4: found node 4470036600
   Shop 5: found node 4470036600
   Shop 6: found node 2904594039
   Shop 7: found node 6651866820
   Shop 8: found node 4470036600
✅ Real road distances calculated!


In [10]:
# Display the distance matrix
print("\n📊 Distance Matrix (km):")
print("     ", end="")
for i in range(len(locations)):
    print(f"{i:6}", end="")
print()

for i in range(len(locations)):
    print(f"{i:2}: ", end="")
    for j in range(len(locations)):
        print(f"{distance_matrix[i][j]:6.1f}", end="")
    print(f"  ({locations[i][0]})")

print(f"\n🚛 Distance calculation method: {'OSMnx Real Roads' if network_success else 'Haversine + 25%'}")


📊 Distance Matrix (km):
          0     1     2     3     4     5     6     7     8
 0:    0.0  37.6  32.2  33.7  38.7  44.9  10.8  23.7  33.3  (Warehouse)
 1:   37.6   0.0   6.9  11.1  11.8  15.0  30.6  25.8  13.3  (Shop 1)
 2:   32.2   6.9   0.0  13.7   9.2  14.7  26.6  24.8   7.7  (Shop 2)
 3:   33.7  11.1  13.7   0.0  22.0  26.0  24.5  16.1  21.4  (Shop 3)
 4:   38.7  11.8   9.2  22.0   0.0   6.2  34.7  33.9   5.7  (Shop 4)
 5:   44.9  15.0  14.7  26.0   6.2   0.0  40.9  39.3  11.7  (Shop 5)
 6:   10.8  30.6  26.6  24.5  34.7  40.9   0.0  13.0  30.1  (Shop 6)
 7:   23.7  25.8  24.8  16.1  33.9  39.3  13.0   0.0  30.9  (Shop 7)
 8:   33.3  13.3   7.7  21.4   5.7  11.7  30.1  30.9   0.0  (Shop 8)

🚛 Distance calculation method: OSMnx Real Roads


In [11]:
# Enhanced OR-Tools preparation with rental vehicle support
SCALE = 100
dist_matrix_int = [[int(round(d * SCALE)) for d in row] for row in distance_matrix]

# Enhanced vehicle specs with full rental information
vehicle_specs = []
vehicle_details = []  # Keep track of original specs for cost calculation

for v in vehicles:
    for _ in range(v["count"]):
        # For OR-Tools compatibility, keep capacity and a basic cost
        vehicle_specs.append((v["capacity"], v.get("cost_per_km", 0)))
        # Store full vehicle specification for enhanced cost calculation
        vehicle_details.append(v)

print(f"\n🔧 Enhanced OR-Tools VRP Setup:")
print(f"   Locations: {len(locations)}")
print(f"   Vehicles: {len(vehicle_specs)}")

for i, (cap, _) in enumerate(vehicle_specs):
    v_detail = vehicle_details[i]
    if v_detail["type"] == "owned":
        print(f"     Vehicle {i}: {cap} bags, OWNED @ ₨{v_detail['cost_per_km']}/km")
    else:
        print(f"     Vehicle {i}: {cap} bags, RENTAL:")
        print(f"       • Daily cost: ₨{v_detail['rental_cost']}")
        print(f"       • Free km: {v_detail['km_limit']}")
        print(f"       • Excess: ₨{v_detail['excess_cost_per_km']}/km")

# Create routing model
manager = pywrapcp.RoutingIndexManager(len(dist_matrix_int), len(vehicle_specs), 0)
routing = pywrapcp.RoutingModel(manager)
print("✅ Enhanced routing model created!")


🔧 Enhanced OR-Tools VRP Setup:
   Locations: 9
   Vehicles: 5
     Vehicle 0: 200 bags, OWNED @ ₨100.0/km
     Vehicle 1: 100 bags, OWNED @ ₨40.0/km
     Vehicle 2: 80 bags, RENTAL:
       • Daily cost: ₨5000.0
       • Free km: 50
       • Excess: ₨80.0/km
     Vehicle 3: 80 bags, RENTAL:
       • Daily cost: ₨5000.0
       • Free km: 50
       • Excess: ₨80.0/km
     Vehicle 4: 50 bags, RENTAL:
       • Daily cost: ₨3000.0
       • Free km: 40
       • Excess: ₨60.0/km
✅ Enhanced routing model created!


In [12]:
# Enhanced cost evaluator for rental and owned vehicles
def create_enhanced_cost_evaluator(vehicle_spec, dist_matrix, manager, scale):
    """Create cost evaluator that handles both owned and rental vehicles"""

    if vehicle_spec["type"] == "owned":
        # Simple cost per km for owned vehicles
        cost_per_km = vehicle_spec["cost_per_km"]
        def owned_cost_eval(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            distance_km = dist_matrix[from_node][to_node] / scale
            return int(round(distance_km * cost_per_km * 100))
        return owned_cost_eval

    else:  # rental vehicle
        rental_cost = vehicle_spec["rental_cost"]
        km_limit = vehicle_spec["km_limit"]
        excess_cost_per_km = vehicle_spec["excess_cost_per_km"]

        def rental_cost_eval(from_index, to_index):
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            distance_km = dist_matrix[from_node][to_node] / scale

            # For routing purposes, we need to estimate cost per segment
            # This is a simplified approach - in reality you'd need route-level tracking
            if distance_km <= km_limit:
                # Within free limit - only rental cost (distributed across route)
                cost = rental_cost / 10  # Distribute fixed cost across potential segments
            else:
                # Exceeds limit - rental cost + excess
                excess_km = distance_km - km_limit
                cost = (rental_cost / 10) + (excess_km * excess_cost_per_km)

            return int(round(cost * 100))
        return rental_cost_eval

# Update vehicle specs with enhanced information
vehicle_specs = []
vehicle_details = []  # Keep track of original specs for later use

for v in vehicles:
    for _ in range(v["count"]):
        vehicle_specs.append((v["capacity"], v.get("cost_per_km", 0)))  # For compatibility
        vehicle_details.append(v)  # Store full vehicle spec

print(f"\n🔧 Enhanced OR-Tools VRP Setup:")
print(f"   Locations: {len(locations)}")
print(f"   Vehicles: {len(vehicle_specs)}")
for i, (cap, _) in enumerate(vehicle_specs):
    v_detail = vehicle_details[i]
    if v_detail["type"] == "owned":
        print(f"     Vehicle {i}: {cap} bags, OWNED @ ₨{v_detail['cost_per_km']}/km")
    else:
        print(f"     Vehicle {i}: {cap} bags, RENTAL (₨{v_detail['rental_cost']}, {v_detail['km_limit']}km limit)")

# Register enhanced cost evaluators for each vehicle
for vid, vehicle_detail in enumerate(vehicle_details):
    evaluator = create_enhanced_cost_evaluator(vehicle_detail, dist_matrix_int, manager, SCALE)
    cost_cb_idx = routing.RegisterTransitCallback(evaluator)
    routing.SetArcCostEvaluatorOfVehicle(cost_cb_idx, vid)

print("✅ Enhanced cost evaluators registered!")



# Add capacity constraints
def demand_callback(from_index):
    node = manager.IndexToNode(from_index)
    return demands[node]

demand_cb_idx = routing.RegisterUnaryTransitCallback(demand_callback)
routing.AddDimensionWithVehicleCapacity(
    demand_cb_idx, 0, [spec[0] for spec in vehicle_specs], True, "Capacity")

# Add fixed cost to minimize vehicle usage
for v in range(len(vehicle_specs)):
    routing.SetFixedCostOfVehicle(1000, v)

print("✅ Constraints and costs added!")


🔧 Enhanced OR-Tools VRP Setup:
   Locations: 9
   Vehicles: 5
     Vehicle 0: 200 bags, OWNED @ ₨100.0/km
     Vehicle 1: 100 bags, OWNED @ ₨40.0/km
     Vehicle 2: 80 bags, RENTAL (₨5000.0, 50km limit)
     Vehicle 3: 80 bags, RENTAL (₨5000.0, 50km limit)
     Vehicle 4: 50 bags, RENTAL (₨3000.0, 40km limit)
✅ Enhanced cost evaluators registered!
✅ Constraints and costs added!


In [13]:
# Solve the VRP
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
search_parameters.time_limit.seconds = 30

print("\n🔍 Solving VRP...")
solution = routing.SolveWithParameters(search_parameters)

if solution:
    print("\n✅ SOLUTION FOUND!")
    print("\n🚛 OPTIMIZED DELIVERY ROUTES:")
    print("=" * 60)

    total_cost = 0
    total_distance = 0
    vehicles_used = 0

    # Store routes for map visualization
    optimized_routes = []

    for v in range(len(vehicle_specs)):
        index = routing.Start(v)
        route = []
        route_locations = []  # Store actual coordinates for mapping
        load = 0
        route_cost = 0
        route_distance = 0

        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(locations[node][0])
            route_locations.append((locations[node][1], locations[node][2]))  # lat, lon
            load += demands[node]
            next_index = solution.Value(routing.NextVar(index))
            route_cost += routing.GetArcCostForVehicle(index, next_index, v)
            if not routing.IsEnd(next_index):
                route_distance += distance_matrix[node][manager.IndexToNode(next_index)]
            index = next_index

        route.append(locations[manager.IndexToNode(index)][0])  # return to warehouse
        route_locations.append((locations[manager.IndexToNode(index)][1], locations[manager.IndexToNode(index)][2]))

        if len(route) > 2:  # vehicle was used
            vehicles_used += 1
            capacity, cost_per_km = vehicle_specs[v]

            # Store route for mapping
            optimized_routes.append({
                'vehicle_id': v + 1,
                'route_names': route,
                'route_coords': route_locations,
                'capacity': capacity,
                'cost_per_km': cost_per_km,
                'load': load,
                'distance': route_distance,
                'cost': route_cost / 100
            })

            print(f"\n🚚 Vehicle {v+1} ({capacity}-bag capacity, ₨{cost_per_km}/km):")
            print(f"   Route: {' → '.join(route)}")
            print(f"   Load: {load}/{capacity} bags ({load/capacity*100:.1f}% full)")
            print(f"   Distance: {route_distance:.1f} km")
            print(f"   Cost: ₨{route_cost/100:.2f}")

            total_cost += route_cost
            total_distance += route_distance

    print(f"\n📊 SUMMARY:")
    print(f"   🚛 Vehicles used: {vehicles_used}/{len(vehicle_specs)}")
    print(f"   📏 Total distance: {total_distance:.1f} km")
    print(f"   💰 Total cost: ₨{total_cost/100:.2f}")
    print(f"   📦 Cost per bag: ₨{total_cost/100/sum(demands):.2f}")
    print(f"   🛣️ Distance method: {'Real OSMnx roads' if network_success else 'Haversine + 25%'}")

else:
    print("❌ No solution found!")
    optimized_routes = []


🔍 Solving VRP...

✅ SOLUTION FOUND!

🚛 OPTIMIZED DELIVERY ROUTES:

🚚 Vehicle 3 (80-bag capacity, ₨0/km):
   Route: Warehouse → Shop 3 → Shop 5 → Warehouse
   Load: 42/80 bags (52.5% full)
   Distance: 59.7 km
   Cost: ₨1510.00

🚚 Vehicle 4 (80-bag capacity, ₨0/km):
   Route: Warehouse → Shop 2 → Shop 1 → Shop 8 → Warehouse
   Load: 80/80 bags (100.0% full)
   Distance: 52.5 km
   Cost: ₨2010.00

🚚 Vehicle 5 (50-bag capacity, ₨0/km):
   Route: Warehouse → Shop 6 → Shop 7 → Shop 4 → Warehouse
   Load: 43/50 bags (86.0% full)
   Distance: 57.7 km
   Cost: ₨1210.00

📊 SUMMARY:
   🚛 Vehicles used: 3/5
   📏 Total distance: 169.8 km
   💰 Total cost: ₨4730.00
   📦 Cost per bag: ₨28.67
   🛣️ Distance method: Real OSMnx roads


In [14]:
vehicles

[{'count': 1, 'capacity': 200, 'type': 'owned', 'cost_per_km': 100.0},
 {'count': 1, 'capacity': 100, 'type': 'owned', 'cost_per_km': 40.0},
 {'count': 2,
  'capacity': 80,
  'type': 'rental',
  'rental_cost': 5000.0,
  'km_limit': 50,
  'excess_cost_per_km': 80.0},
 {'count': 1,
  'capacity': 50,
  'type': 'rental',
  'rental_cost': 3000.0,
  'km_limit': 40,
  'excess_cost_per_km': 60.0}]

In [15]:
# Create interactive map with optimized routes
import folium
from folium import plugins

def create_route_map(locations, optimized_routes, network_graph=None):
    """Create folium map showing optimized delivery routes"""

    # Calculate center of map
    center_lat = sum(loc[1] for loc in locations) / len(locations)
    center_lon = sum(loc[2] for loc in locations) / len(locations)

    # Create base map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=12,
        tiles='OpenStreetMap'
    )

    # Colors for different vehicles
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred', 'lightred',
             'beige', 'darkblue', 'darkgreen', 'cadetblue', 'darkpurple', 'white',
             'pink', 'lightblue', 'lightgreen', 'gray', 'black', 'lightgray']

    # Add warehouse marker (larger, special icon)
    warehouse = locations[0]
    folium.Marker(
        location=[warehouse[1], warehouse[2]],
        popup=f"🏭 {warehouse[0]}<br>Main Depot",
        tooltip=f"{warehouse[0]} (Depot)",
        icon=folium.Icon(color='black', icon='home', prefix='fa')
    ).add_to(m)

    # Add shop markers
    for i, (name, lat, lon) in enumerate(locations[1:], 1):
        demand = demands[i]
        folium.Marker(
            location=[lat, lon],
            popup=f"🏪 {name}<br>Demand: {demand} bags",
            tooltip=f"{name}: {demand} bags",
            icon=folium.Icon(color='lightblue', icon='shopping-cart', prefix='fa')
        ).add_to(m)

    # Add optimized routes
    for i, route in enumerate(optimized_routes):
        color = colors[i % len(colors)]

        # Create route line
        route_coords = route['route_coords']

        if network_graph is not None and network_success:
            # If we have real road network, get actual route paths
            try:
                full_route_coords = []
                for j in range(len(route_coords) - 1):
                    start_lat, start_lon = route_coords[j]
                    end_lat, end_lon = route_coords[j + 1]

                    # Get nearest nodes
                    start_node = ox.nearest_nodes(network_graph, start_lon, start_lat)
                    end_node = ox.nearest_nodes(network_graph, end_lon, end_lat)

                    # Get shortest path
                    path = nx.shortest_path(network_graph, start_node, end_node, weight='length')

                    # Get path coordinates
                    path_coords = [(network_graph.nodes[node]['y'], network_graph.nodes[node]['x']) for node in path]
                    full_route_coords.extend(path_coords)

                # Draw detailed route
                folium.PolyLine(
                    locations=full_route_coords,
                    color=color,
                    weight=4,
                    opacity=0.8,
                    popup=f"Vehicle {route['vehicle_id']}: {route['distance']:.1f}km, ₨{route['cost']:.2f}"
                ).add_to(m)

            except Exception as e:
                print(f"⚠️ Could not draw detailed route for Vehicle {route['vehicle_id']}: {e}")
                # Fallback to straight lines
                folium.PolyLine(
                    locations=route_coords,
                    color=color,
                    weight=3,
                    opacity=0.7,
                    popup=f"Vehicle {route['vehicle_id']}: {route['distance']:.1f}km, ₨{route['cost']:.2f}"
                ).add_to(m)
        else:
            # Draw straight-line routes
            folium.PolyLine(
                locations=route_coords,
                color=color,
                weight=3,
                opacity=0.7,
                popup=f"Vehicle {route['vehicle_id']}: {route['distance']:.1f}km, ₨{route['cost']:.2f}"
            ).add_to(m)

        # Add route info to legend
        route_info = f"""
        <div style='position: fixed;
                    top: {50 + i*25}px; left: 50px; width: 300px; height: 20px;
                    background-color: white; border:2px solid grey; z-index:9999;
                    font-size:12px; padding: 5px'>
        <p style='color: {color}; margin:0'>
        ● Vehicle {route['vehicle_id']}: {route['load']}/{route['capacity']} bags,
        {route['distance']:.1f}km, ₨{route['cost']:.2f}
        </p>
        </div>
        """
        m.get_root().html.add_child(folium.Element(route_info))

    # Add map title
    title_html = '''
                 <h3 align="center" style="font-size:16px"><b>🚛 Optimized Delivery Routes - Lahore VRP Solution</b></h3>
                 '''
    m.get_root().html.add_child(folium.Element(title_html))

    return m

# Create and display the map
if optimized_routes:
    print("\n🗺️ Creating interactive route map...")
    route_map = create_route_map(locations, optimized_routes, G if network_success else None)

    # Save map
    map_filename = "bhawana_vrp_routes.html"
    route_map.save(map_filename)
    print(f"✅ Map saved as: {map_filename}")

    # Display map in notebook
    route_map
else:
    print("❌ No routes to display on map")


🗺️ Creating interactive route map...
✅ Map saved as: bhawana_vrp_routes.html


In [16]:
# Enhanced solution display with rental cost breakdown
if solution:
    print("\n✅ SOLUTION FOUND!")
    print("\n🚛 OPTIMIZED DELIVERY ROUTES:")
    print("=" * 70)

    total_cost = 0
    total_distance = 0
    vehicles_used = 0
    optimized_routes = []

    for v in range(len(vehicle_specs)):
        index = routing.Start(v)
        route = []
        route_locations = []
        load = 0
        route_cost = 0
        route_distance = 0

        while not routing.IsEnd(index):
            node = manager.IndexToNode(index)
            route.append(locations[node][0])
            route_locations.append((locations[node][1], locations[node][2]))
            load += demands[node]
            next_index = solution.Value(routing.NextVar(index))
            route_cost += routing.GetArcCostForVehicle(index, next_index, v)
            if not routing.IsEnd(next_index):
                route_distance += distance_matrix[node][manager.IndexToNode(next_index)]
            index = next_index

        route.append(locations[manager.IndexToNode(index)][0])
        route_locations.append((locations[manager.IndexToNode(index)][1], locations[manager.IndexToNode(index)][2]))

        if len(route) > 2:  # vehicle was used
            vehicles_used += 1
            capacity = vehicle_specs[v][0]
            vehicle_detail = vehicle_details[v]

            # Calculate actual costs based on vehicle type
            if vehicle_detail["type"] == "owned":
                actual_cost = route_distance * vehicle_detail["cost_per_km"]
                cost_breakdown = f"₨{actual_cost:.2f} ({route_distance:.1f}km × ₨{vehicle_detail['cost_per_km']}/km)"
            else:
                rental_cost = vehicle_detail["rental_cost"]
                km_limit = vehicle_detail["km_limit"]
                excess_cost_per_km = vehicle_detail["excess_cost_per_km"]

                if route_distance <= km_limit:
                    actual_cost = rental_cost
                    cost_breakdown = f"₨{actual_cost:.2f} (rental only, within {km_limit}km limit)"
                else:
                    excess_km = route_distance - km_limit
                    excess_cost = excess_km * excess_cost_per_km
                    actual_cost = rental_cost + excess_cost
                    cost_breakdown = f"₨{actual_cost:.2f} (₨{rental_cost} rental + ₨{excess_cost:.2f} for {excess_km:.1f}km excess)"

            vehicle_type_str = "OWNED" if vehicle_detail["type"] == "owned" else "RENTAL"

            optimized_routes.append({
                'vehicle_id': v + 1,
                'route_names': route,
                'route_coords': route_locations,
                'capacity': capacity,
                'vehicle_type': vehicle_type_str,
                'load': load,
                'distance': route_distance,
                'cost': actual_cost,
                'cost_breakdown': cost_breakdown
            })

            print(f"\n🚚 Vehicle {v+1} ({capacity}-bag {vehicle_type_str}):")
            print(f"   Route: {' → '.join(route)}")
            print(f"   Load: {load}/{capacity} bags ({load/capacity*100:.1f}% full)")
            print(f"   Distance: {route_distance:.1f} km")
            print(f"   Cost: {cost_breakdown}")

            total_cost += actual_cost
            total_distance += route_distance

    print(f"\n📊 ENHANCED SUMMARY:")
    print(f"   🚛 Vehicles used: {vehicles_used}/{len(vehicle_specs)}")
    print(f"   📏 Total distance: {total_distance:.1f} km")
    print(f"   💰 Total cost: ₨{total_cost:.2f}")
    print(f"   📦 Cost per bag: ₨{total_cost/sum(demands):.2f}")
    print(f"   🛣️ Distance method: {'Real OSMnx roads' if network_success else 'Haversine + 25%'}")

    # Vehicle type breakdown
    owned_cost = sum(r['cost'] for r in optimized_routes if r['vehicle_type'] == 'OWNED')
    rental_cost = sum(r['cost'] for r in optimized_routes if r['vehicle_type'] == 'RENTAL')

    print(f"\n💰 Cost Breakdown by Vehicle Type:")
    print(f"   • Owned vehicles: ₨{owned_cost:.2f}")
    print(f"   • Rental vehicles: ₨{rental_cost:.2f}")

else:
    print("❌ No solution found!")
    optimized_routes = []


✅ SOLUTION FOUND!

🚛 OPTIMIZED DELIVERY ROUTES:

🚚 Vehicle 3 (80-bag RENTAL):
   Route: Warehouse → Shop 3 → Shop 5 → Warehouse
   Load: 42/80 bags (52.5% full)
   Distance: 59.7 km
   Cost: ₨5774.39 (₨5000.0 rental + ₨774.39 for 9.7km excess)

🚚 Vehicle 4 (80-bag RENTAL):
   Route: Warehouse → Shop 2 → Shop 1 → Shop 8 → Warehouse
   Load: 80/80 bags (100.0% full)
   Distance: 52.5 km
   Cost: ₨5196.55 (₨5000.0 rental + ₨196.55 for 2.5km excess)

🚚 Vehicle 5 (50-bag RENTAL):
   Route: Warehouse → Shop 6 → Shop 7 → Shop 4 → Warehouse
   Load: 43/50 bags (86.0% full)
   Distance: 57.7 km
   Cost: ₨4060.05 (₨3000.0 rental + ₨1060.05 for 17.7km excess)

📊 ENHANCED SUMMARY:
   🚛 Vehicles used: 3/5
   📏 Total distance: 169.8 km
   💰 Total cost: ₨15030.99
   📦 Cost per bag: ₨91.10
   🛣️ Distance method: Real OSMnx roads

💰 Cost Breakdown by Vehicle Type:
   • Owned vehicles: ₨0.00
   • Rental vehicles: ₨15030.99



🚚 Vehicle 1 (200-bag OWNED):
   Route: Warehouse → Shop 7 → Shop 6 → Warehouse
   Load: 33/200 bags (16.5% full)
   Distance: 36.7 km
   Cost: ₨3669.41 (36.7km × ₨100.0/km)

🚚 Vehicle 2 (100-bag OWNED):
   Route: Warehouse → Shop 18 → Shop 14 → Shop 3 → Shop 1 → Shop 2 → Shop 11 → Shop 19 → Shop 16 → Shop 17 → Warehouse
   Load: 90/100 bags (90.0% full)
   Distance: 96.1 km
   Cost: ₨3845.95 (96.1km × ₨40.0/km)

🚚 Vehicle 5 (50-bag RENTAL):
   Route: Warehouse → Shop 9 → Shop 20 → Shop 4 → Shop 5 → Shop 8 → Shop 13 → Shop 12 → Shop 10 → Shop 15 → Warehouse
   Load: 42/50 bags (84.0% full)
   Distance: 188.1 km
   Cost: ₨11885.16 (₨3000.0 rental + ₨8885.16 for 148.1km excess)

📊 ENHANCED SUMMARY:
   🚛 Vehicles used: 3/5
   📏 Total distance: 320.9 km
   💰 Total cost: ₨19400.53
   📦 Cost per bag: ₨117.58
   🛣️ Distance method: Real OSMnx roads

💰 Cost Breakdown by Vehicle Type:
   • Owned vehicles: ₨7515.37
   • Rental vehicles: ₨11885.16